In [11]:
# Libraries
from pypdf import PdfReader, PdfWriter, Transformation
import camelot
import pandas as pd
import os
import glob

In [12]:
# Functions

# Define a function to extract specific pages from a PDF file
def extract_pdf(source_path, new_pdf, start_page, end_page):
    """
    Splits a PDF into a new PDF containing only the specified page range.
    Creates the output directory if it does not exist.

    Parameters:
        source_path (str): Path to the input PDF file.
        new_pdf (str): Path to save the output PDF file.
        start_page (int): Starting page number (inclusive, 1-based).
        end_page (int): Ending page number (inclusive, 1-based).
    """
    output_dir = os.path.dirname(new_pdf)
    os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists

    reader = PdfReader(source_path)
    writer = PdfWriter()

    for page_num in range(start_page - 1, end_page):
        writer.add_page(reader.pages[page_num])

    with open(new_pdf, "wb") as f:
        writer.write(f)

    print(f"Saved new PDF to {new_pdf} with pages {start_page}-{end_page}")

# Define a function to crop odd and even pages to specific respective dimensions.
def split_and_crop(input_path, output_path, odd_crop, even_crop):
    """
    Crops odd and even pages of a PDF with different dimensions and saves the result.

    Parameters:
        input_path (str): Path to the input PDF file.
        output_path (str): Full path to save the output PDF file.
        odd_crop (dict): Crop dimensions for odd pages.
        even_crop (dict): Crop dimensions for even pages.
    """
    output_dir = os.path.dirname(output_path)
    os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists

    reader = PdfReader(input_path)
    writer = PdfWriter()

    for i, page in enumerate(reader.pages):
        mediabox = page.mediabox
        crop = odd_crop if i % 2 == 0 else even_crop
        mediabox.lower_left = (mediabox.lower_left[0] + crop["left"], mediabox.lower_left[1] + crop["bottom"])
        mediabox.upper_right = (mediabox.upper_right[0] - crop["right"], mediabox.upper_right[1] - crop["top"])
        writer.add_page(page)

    with open(output_path, "wb") as f:
        writer.write(f)

    print(f"Cropped PDF saved to {output_path}")

# Function to split each page vertically in half
# NOT USING FOR NOW. KEPT FOR FUTURE REFERENCE.
def split_pages_vertically(input_path, output_path):
    """
    Splits each page of a PDF vertically in half, creating two pages from each original page.
    
    Parameters:
        input_path (str): Path to the input PDF file.
        output_path (str): Full path to save the output PDF file.
    """
    output_dir = os.path.dirname(output_path)
    os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists

    reader = PdfReader(input_path)
    writer = PdfWriter()

    for page in reader.pages:
        # Get the original page dimensions
        mediabox = page.mediabox
        width = mediabox.width
        height = mediabox.height
        half_width = width / 2
        
        # Create left half (first column) - show left side of original page
        left_page = writer.add_blank_page(width=half_width, height=height)
        left_page.merge_transformed_page(
            page,
            Transformation().translate(tx=0, ty=0)  # No translation for left half
        )
        # Crop to show only left half
        left_page.mediabox.lower_left = (0, 0)
        left_page.mediabox.upper_right = (half_width, height)
        
        # Create right half (second column) - show right side of original page
        right_page = writer.add_blank_page(width=half_width, height=height)
        right_page.merge_transformed_page(
            page,
            Transformation().translate(tx=-half_width, ty=0)  # Shift left by half width
        )
        # Crop to show only right half
        right_page.mediabox.lower_left = (0, 0)
        right_page.mediabox.upper_right = (half_width, height)

    with open(output_path, "wb") as f:
        writer.write(f)

    print(f"Vertically split PDF saved to {output_path}")

# Define a function to extract tables.
def extract_tables_to_csv(input_pdf, output_dir, flavor='stream', strip_text='\n'):
    """
    Extracts tables from a PDF using Camelot and saves each table as a separate CSV file.
    Creates the output directory if it does not exist.

    Parameters:
        input_pdf (str): Path to the input PDF file.
        output_dir (str): Directory to save the CSV files.
        flavor (str): Camelot extraction flavor ('stream' or 'lattice').
        strip_text (str): Characters to strip from extracted text.
    """
   
    os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists

    tables = camelot.read_pdf(input_pdf, pages='all', flavor=flavor, strip_text=strip_text)
    print(f"Found {tables.n} tables")
    for i, table in enumerate(tables):
        output_csv = os.path.join(output_dir, f"table_{i+1}.csv")  # Cross-platform path
        table.df.to_csv(output_csv, index=False)
        print(f"Saved table {i+1} to {output_csv}")

In [13]:
# define paths
source_path = r"C:\Users\alex\coding_projects\bls_data\data\source\workstoppage"
output_dir = r"C:\Users\alex\coding_projects\bls_data\data\output\workstoppage"

# scan the source directory for PDF files
pdf_files = sorted(glob.glob(os.path.join(source_path, "*.pdf")))
pdf_list = []

print("PDF files found in source directory:")
print("-" * 50)
for i, pdf_path in enumerate(pdf_files, start=1):
    pdf_name = os.path.basename(pdf_path)
    pdf_list.append(pdf_name)
    print(f"{i:2d}. {pdf_name}")

print("-" * 50)
print(f"Total PDFs found: {len(pdf_files)}")
print("\nYou can now reference these PDFs by their number in the pdf_configs list below.\n")

PDF files found in source directory:
--------------------------------------------------
 1. work-stoppages-1950.pdf
 2. work-stoppages-1951.pdf
 3. work-stoppages-1952.pdf
 4. work-stoppages-1953.pdf
 5. work-stoppages-1954.pdf
 6. work-stoppages-1955.pdf
 7. work-stoppages-1956.pdf
 8. work-stoppages-1957.pdf
 9. work-stoppages-1958.pdf
10. work-stoppages-1959.pdf
11. work-stoppages-1960.pdf
12. work-stoppages-1961.pdf
13. work-stoppages-1962.pdf
14. work-stoppages-1963.pdf
15. work-stoppages-1964.pdf
16. work-stoppages-1965.pdf
17. work-stoppages-1966.pdf
18. work-stoppages-1967.pdf
19. work-stoppages-1968.pdf
20. work-stoppages-1969.pdf
21. work-stoppages-1970.pdf
22. work-stoppages-1971.pdf
23. work-stoppages-1972.pdf
24. work-stoppages-1973.pdf
25. work-stoppages-1974.pdf
--------------------------------------------------
Total PDFs found: 25

You can now reference these PDFs by their number in the pdf_configs list below.



In [14]:
# Define your PDF configurations using the numbers from the list above
pdf_configs = [
    {
        "pdf_number": 1,  # Reference the PDF by its number from the list above
        "output_name": "1950",
        "start_page": 11,
        "end_page": 12,
        "odd_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0},
        "even_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0}
    },
    {
        "pdf_number": 2,
        "output_name": "1951",
        "start_page": 15,
        "end_page": 16,
        "odd_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0},
        "even_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0}
    },
    {
        "pdf_number": 3,
        "output_name": "1952", 
        "start_page": 13,
        "end_page": 14,
        "odd_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0},
        "even_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0}
    },
    {
        "pdf_number": 4,  
        "output_name": "1953", 
        "start_page": 12,
        "end_page": 13,
        "odd_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0},
        "even_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0}
    },
    {
        "pdf_number": 5,  
        "output_name": "1954", 
        "start_page": 15,
        "end_page": 17,
        "odd_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0},
        "even_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0}
    },
    {
        "pdf_number": 6,  
        "output_name": "1955", 
        "start_page": 10,
        "end_page": 12,
        "odd_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0},
        "even_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0}
    },
    {
        "pdf_number": 7,  
        "output_name": "1956", 
        "start_page": 14,
        "end_page": 17,
        "odd_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0},
        "even_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0}
    },
    {
        "pdf_number": 8,  
        "output_name": "1957", 
        "start_page": 14,
        "end_page": 16,
        "odd_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0},
        "even_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0}
    },
    {
        "pdf_number": 9,  
        "output_name": "1958", 
        "start_page": 12,
        "end_page": 14,
        "odd_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0},
        "even_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0}
    },
    {
        "pdf_number": 10,  
        "output_name": "1959", 
        "start_page": 13,
        "end_page": 15,
        "odd_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0},
        "even_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0}
    },
    {
        "pdf_number": 11,  
        "output_name": "1960", 
        "start_page": 11,
        "end_page": 13,
        "odd_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0},
        "even_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0}
    },
    {
        "pdf_number": 12,  
        "output_name": "1961", 
        "start_page": 8,
        "end_page": 17,
        "odd_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0},
        "even_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0}
    },
    {
        "pdf_number": 13,  
        "output_name": "1962", 
        "start_page": 8,
        "end_page": 17,
        "odd_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0},
        "even_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0}
    },
    {
        "pdf_number": 14,  
        "output_name": "1963", 
        "start_page": 8,
        "end_page": 17,
        "odd_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0},
        "even_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0}
    },
    {
        "pdf_number": 15,  
        "output_name": "1964", 
        "start_page": 8,
        "end_page": 17,
        "odd_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0},
        "even_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0}
    },
    {
        "pdf_number": 16,  
        "output_name": "1965", 
        "start_page": 8,
        "end_page": 17,
        "odd_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0},
        "even_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0}
    },
    {
        "pdf_number": 17,  
        "output_name": "1966", 
        "start_page": 8,
        "end_page": 17,
        "odd_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0},
        "even_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0}
    },
    {
        "pdf_number": 18,  
        "output_name": "1967", 
        "start_page": 8,
        "end_page": 17,
        "odd_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0},
        "even_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0}
    },
    {
        "pdf_number": 19,  
        "output_name": "1968", 
        "start_page": 8,
        "end_page": 17,
        "odd_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0},
        "even_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0}
    },
    {
        "pdf_number": 20,  
        "output_name": "1969", 
        "start_page": 8,
        "end_page": 17,
        "odd_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0},
        "even_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0}
    },
    {
        "pdf_number": 21,  
        "output_name": "1970", 
        "start_page": 8,
        "end_page": 17,
        "odd_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0},
        "even_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0}
    },
    {
        "pdf_number": 22,  
        "output_name": "1971", 
        "start_page": 8,
        "end_page": 17,
        "odd_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0},
        "even_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0}
    },
    {
        "pdf_number": 23,  
        "output_name": "1972", 
        "start_page": 8,
        "end_page": 17,
        "odd_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0},
        "even_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0}
    },
    {
        "pdf_number": 24,  
        "output_name": "1973", 
        "start_page": 8,
        "end_page": 17,
        "odd_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0},
        "even_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0}
    },
    {
        "pdf_number": 25,  
        "output_name": "1974", 
        "start_page": 8,
        "end_page": 17,
        "odd_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0},
        "even_crop": {"left": 0, "bottom": 0, "right": 0, "top": 0}
    }
    # Add more configurations as needed
]

In [15]:
# Process each PDF configuration
for idx, config in enumerate(pdf_configs, start=1):
    try:
        # Get the PDF path based on the number (FIXED)
        pdf_number = config["pdf_number"]
        if pdf_number < 1 or pdf_number > len(pdf_files):
            print(f"ERROR: PDF number {pdf_number} is out of range. Available: 1-{len(pdf_files)}")
            continue
            
        pdf_path = pdf_files[pdf_number - 1]  # Convert to 0-based index
        pdf_name = os.path.basename(pdf_path)
        
        # Create output directory
        pdf_output_dir = os.path.join(output_dir, f"{idx:02d}_{config['output_name']}")
        os.makedirs(pdf_output_dir, exist_ok=True)

        print(f"\nProcessing #{pdf_number}: {pdf_name} (pages {config['start_page']}-{config['end_page']})...")

        # Step 1: Extract the specific pages
        extracted_pdf_path = os.path.join(pdf_output_dir, f"{config['output_name']}_extracted.pdf")
        extract_pdf(
            pdf_path,  # Use the actual path, not config["input_pdf"]
            extracted_pdf_path,
            config["start_page"],
            config["end_page"]
        )

        # Step 2: Crop the pages
        cropped_pdf_path = os.path.join(pdf_output_dir, f"{config['output_name']}_cropped.pdf")
        split_and_crop(
            extracted_pdf_path,
            cropped_pdf_path,
            config["odd_crop"],
            config["even_crop"]
        )

        # NEW: Step 3: Split each page vertically in half
        #split_pdf_path = os.path.join(pdf_output_dir, f"{config['output_name']}_split.pdf")
        #split_pages_vertically(
         #   cropped_pdf_path,
          #  split_pdf_path
        #)

        # Step 4: Extract tables to CSV (now from the split PDF)
        extract_tables_to_csv(
            #split_pdf_path,  # Changed from cropped_pdf_path to split_pdf_path
            cropped_pdf_path,
            pdf_output_dir
        )
        
        print(f"Successfully processed {pdf_name}\n")
    
    except FileNotFoundError as e:
        print(f"ERROR: Could not find file: {e}")
        continue
    except Exception as e:
        print(f"ERROR: Failed to process PDF #{config.get('pdf_number', 'unknown')}. Reason: {e}")
        continue


Processing #1: work-stoppages-1950.pdf (pages 11-12)...
Saved new PDF to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\01_1950\1950_extracted.pdf with pages 11-12
Cropped PDF saved to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\01_1950\1950_cropped.pdf
Found 4 tables
Saved table 1 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\01_1950\table_1.csv
Saved table 2 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\01_1950\table_2.csv
Saved table 3 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\01_1950\table_3.csv
Saved table 4 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\01_1950\table_4.csv
Successfully processed work-stoppages-1950.pdf


Processing #2: work-stoppages-1951.pdf (pages 15-16)...
Saved new PDF to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\02_1951\1951_extracted.pdf with pages 15-16
Cropped PDF saved to C:\Users\alex\coding_projects\bls_data

C:\Users\alex\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\camelot\parsers\base.py:238: UserWarning: No tables found in table area (0, 0, 460.0, 378.0)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)


Saved new PDF to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\09_1958\1958_extracted.pdf with pages 12-14
Cropped PDF saved to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\09_1958\1958_cropped.pdf
Found 4 tables
Saved table 1 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\09_1958\table_1.csv
Saved table 2 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\09_1958\table_2.csv
Saved table 3 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\09_1958\table_3.csv
Saved table 4 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\09_1958\table_4.csv
Successfully processed work-stoppages-1958.pdf


Processing #10: work-stoppages-1959.pdf (pages 13-15)...
Saved new PDF to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\10_1959\1959_extracted.pdf with pages 13-15
Cropped PDF saved to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\10_1959\1959_cropped.pdf
Found

C:\Users\alex\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\camelot\parsers\base.py:238: UserWarning: No tables found in table area (276.1, 187.696, 521.7371999999999, 346.62208333333336)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)
C:\Users\alex\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\camelot\parsers\base.py:238: UserWarning: No tables found in table area (276.1, 9.762, 520.3552400000001, 139.27084374999998)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)


Found 18 tables
Saved table 1 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\15_1964\table_1.csv
Saved table 2 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\15_1964\table_2.csv
Saved table 3 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\15_1964\table_3.csv
Saved table 4 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\15_1964\table_4.csv
Saved table 5 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\15_1964\table_5.csv
Saved table 6 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\15_1964\table_6.csv
Saved table 7 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\15_1964\table_7.csv
Saved table 8 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\15_1964\table_8.csv
Saved table 9 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\15_1964\table_9.csv
Saved table 10 to C:\Users\alex\coding_projects\bls_data\data\output\workst

C:\Users\alex\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\camelot\parsers\base.py:238: UserWarning: No tables found in table area (242.25, 90.879, 487.56899999999996, 303.5141826923077)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)
C:\Users\alex\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\camelot\parsers\base.py:238: UserWarning: No tables found in table area (262.149, 68.33, 507.1022, 258.12439252336446)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)
C:\Users\alex\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\camelot\parsers\base.py:238: UserWarning: No tables found in table area (1.5, 335.08, 246.59360000000004, 548.3815714285714)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, use

Found 21 tables
Saved table 1 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\21_1970\table_1.csv
Saved table 2 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\21_1970\table_2.csv
Saved table 3 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\21_1970\table_3.csv
Saved table 4 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\21_1970\table_4.csv
Saved table 5 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\21_1970\table_5.csv
Saved table 6 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\21_1970\table_6.csv
Saved table 7 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\21_1970\table_7.csv
Saved table 8 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\21_1970\table_8.csv
Saved table 9 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\21_1970\table_9.csv
Saved table 10 to C:\Users\alex\coding_projects\bls_data\data\output\workst

C:\Users\alex\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\camelot\parsers\base.py:238: UserWarning: No tables found in table area (21.199, 477.529, 268.359, 594.5701960784314)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)


Found 18 tables
Saved table 1 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\22_1971\table_1.csv
Saved table 2 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\22_1971\table_2.csv
Saved table 3 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\22_1971\table_3.csv
Saved table 4 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\22_1971\table_4.csv
Saved table 5 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\22_1971\table_5.csv
Saved table 6 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\22_1971\table_6.csv
Saved table 7 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\22_1971\table_7.csv
Saved table 8 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\22_1971\table_8.csv
Saved table 9 to C:\Users\alex\coding_projects\bls_data\data\output\workstoppage\22_1971\table_9.csv
Saved table 10 to C:\Users\alex\coding_projects\bls_data\data\output\workst